In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 21: Chronal Conversion ---</h2><p>You should have been watching where you were going, because as you wander the new North Pole base, you trip and fall into a very deep hole!</p>
<p><span title="The old time travel hole gag! Classic.">Just kidding.</span>  You're falling through time again.</p>
<p>If you keep up your current pace, you should have resolved all of the temporal anomalies by the next time the device activates. Since you have very little interest in browsing history in 500-year increments for the rest of your life, you need to find a way to get back to your present time.</p>
<p>After a little research, you discover two important facts about the behavior of the device:</p>
<p>First, you discover that the device is hard-wired to always send you back in time in 500-year increments. Changing this is probably not feasible.</p>
<p>Second, you discover the <em>activation system</em> (your puzzle input) for the time travel module.  Currently, it appears to <em>run forever without halting</em>.</p>
<p>If you can cause the activation system to <em>halt</em> at a specific moment, maybe you can make the device send you so far back in time that you cause an <a href="https:#cwe.mitre.org/data/definitions/191.html">integer underflow</a> <em>in time itself</em> and wrap around back to your current time!</p>
<p>The device executes the program as specified in <a href="16">manual section one</a> and <a href="19">manual section two</a>.</p>
<p>Your goal is to figure out how the program works and cause it to halt.  You can only control <em>register <code>0</code></em> every other register begins at <code>0</code> as usual.</p>
<p>Because time travel is a dangerous activity, the activation system begins with a few instructions which verify that <em>bitwise AND</em> (via <code>bani</code>) does a <em>numeric</em> operation and <em>not</em> an operation as if the inputs were interpreted as strings. If the test fails, it enters an infinite loop re-running the test instead of allowing the program to execute normally.  If the test passes, the program continues, and assumes that <em>all other bitwise operations</em> (<code>banr</code>, <code>bori</code>, and <code>borr</code>) also interpret their inputs as <em>numbers</em>. (Clearly, the Elves who wrote this system were worried that someone might introduce a bug while trying to emulate this system with a scripting language.)</p>
<p><em>What is the lowest non-negative integer value for register <code>0</code> that causes the program to halt after executing the fewest instructions?</em> (Executing the same instruction multiple times counts as multiple instructions executed.)</p>
</article>


In [47]:
from math import floor
from re import match

from more_itertools import last, tail


puzzle = """
#ip 2
seti 123 0 5
bani 5 456 5
eqri 5 72 5
addr 5 2 2
seti 0 0 2
seti 0 4 5
bori 5 65536 1
seti 10678677 3 5
bani 1 255 4
addr 5 4 5
bani 5 16777215 5
muli 5 65899 5
bani 5 16777215 5
gtir 256 1 4
addr 4 2 2
addi 2 1 2
seti 27 5 2
seti 0 6 4
addi 4 1 3
muli 3 256 3
gtrr 3 1 3
addr 3 2 2
addi 2 1 2
seti 25 4 2
addi 4 1 4
seti 17 6 2
setr 4 6 1
seti 7 5 2
eqrr 5 0 4
addr 4 2 2
seti 5 4 2
"""

puzzle_optimizes = """
#ip 2
seti 123 0 5
bani 5 456 5
eqri 5 72 5
addr 5 2 2
seti 0 0 2
addi 2 -256 2  
divi 2 256 2   
addi 2 1 2     
noop 2 3 4       
noop 2 3 4          
bani 5 16777215 5
muli 5 65899 5
bani 5 16777215 5
gtir 256 1 4
addr 4 2 2
addi 2 1 2
seti 27 5 2
seti 0 6 4
addi 4 1 3
muli 3 256 3
gtrr 3 1 3
addr 3 2 2
addi 2 1 2
seti 25 4 2
addi 4 1 4
seti 17 6 2
setr 4 6 1
seti 7 5 2
eqrr 5 0 4
addr 4 2 2
seti 5 4 2
"""


class Device:
    def __init__(self, value_rigister_0: int = 0, number_of_registers: int = 6) -> None:
        self.ip = 0
        self.registers = [0] * number_of_registers
        self.registers[0] = value_rigister_0
        self.instructions = []

    def load(self, program: str) -> Device:
        self.instructions = [
            [int(d) if match(r"-?\d+", d) else d for d in line.split()]
            for line in program.strip().splitlines()
        ]
        self.ip = last(self.instructions.pop(0))

        return self

    def process(self, do_print: bool = False) -> Device:
        self.step_count = 0
        while 0 <= self.registers[self.ip] < len(self.instructions):
            self.process_line(self.registers[self.ip], do_print)
            self.registers[self.ip] += 1
            self.step_count += 1
        self.registers[self.ip] -= 1
        return self

    def process_and_print_out_register_at_line(
        self, line: int, register: int, return_first_time: bool = True
    ) -> Device:
        # 15 minutes way too slow
        seen = set()
        all = []
        self.step_count = 0
        while 0 <= self.registers[self.ip] < len(self.instructions):
            if self.registers[self.ip] == line:
                if return_first_time:
                    return self
                else:
                    # print(f"{line=} register[{register}]={self.registers[register]}")
                    if self.registers[register] in seen:
                        print(
                            f"{line=} register[{register}]={self.registers[register]}"
                        )
                        print(all[-1])
                        return self
                    seen.add(self.registers[register])
                    all.append(self.registers[register])

            self.process_line(self.registers[self.ip], False)
            self.registers[self.ip] += 1
            self.step_count += 1
        self.registers[self.ip] -= 1
        return self

    def process_line(self, ip: int, do_print: bool) -> None:
        inst, A, B, C = self.instructions[ip]
        # Addition:
        # addr (add register) stores into register C the result of
        # adding register A and register B.
        if do_print:
            s = f"{self} {inst} {A} {B} {C}"

        if inst == "addr":
            self.registers[C] = self.registers[A] + self.registers[B]
        # addi (add immediate) stores into register C the result of
        # adding register A and value B.
        elif inst == "addi":
            self.registers[C] = self.registers[A] + B

        # Multiplication:
        # mulr (multiply register) stores into register C the result
        # of multiplying register A and register B.
        elif inst == "mulr":
            self.registers[C] = self.registers[A] * self.registers[B]
        # muli (multiply immediate) stores into register C the result
        # of multiplying register A and value B.
        elif inst == "muli":
            self.registers[C] = self.registers[A] * B

        # Bitwise AND:
        # banr (bitwise AND register) stores into register C the result
        # of the bitwise AND of register A and register B.
        elif inst == "banr":
            self.registers[C] = self.registers[A] & self.registers[B]
        # bani (bitwise AND immediate) stores into register C the result
        # of the bitwise AND of register A and value B.
        elif inst == "bani":
            self.registers[C] = self.registers[A] & B

        # Bitwise OR:
        # borr (bitwise OR register) stores into register C the result
        # of the bitwise OR of register A and register B.
        elif inst == "borr":
            self.registers[C] = self.registers[A] | self.registers[B]
        # bori (bitwise OR immediate) stores into register C the result
        # of the bitwise OR of register A and value B.
        elif inst == "bori":
            self.registers[C] = self.registers[A] | B

        # Assignment:
        # setr (set register) copies the contents of register A into register C.
        # (Input B is ignored.)
        elif inst == "setr":
            self.registers[C] = self.registers[A]
        # seti (set immediate) stores value A into register C. (Input B is ignored.)
        elif inst == "seti":
            self.registers[C] = A

        # Greater-than testing:
        # gtir (greater-than immediate/register)
        # sets register C to 1 if value A is greater than register B.
        # Otherwise, register C is set to 0.
        elif inst == "gtir":
            self.registers[C] = 1 if A > self.registers[B] else 0

        # gtri (greater-than register/immediate) sets register C to 1
        # if register A is greater than value B. Otherwise, register C is set to 0.
        elif inst == "gtri":
            self.registers[C] = 1 if self.registers[A] > B else 0
        # gtrr (greater-than register/register) sets register C to 1
        # if register A is greater than register B. Otherwise, register C
        # is set to 0.
        elif inst == "gtrr":
            self.registers[C] = 1 if self.registers[A] > self.registers[B] else 0

        # Equality testing:
        # eqir (equal immediate/register) sets register C to 1
        # if value A is equal to register B. Otherwise, register C is set to 0.
        elif inst == "eqir":
            self.registers[C] = 1 if A == self.registers[B] else 0
        # eqri (equal register/immediate) sets register C to 1
        # if register A is equal to value B. Otherwise, register C is set to 0.
        elif inst == "eqri":
            self.registers[C] = 1 if self.registers[A] == B else 0
        # eqrr (equal register/register) sets register C to 1
        # if register A is equal to register B. Otherwise, register C is set to 0.
        elif inst == "eqrr":
            self.registers[C] = 1 if self.registers[A] == self.registers[B] else 0

        if do_print:
            print(f"{s} {self.registers}")

    def stringify_program(self) -> str:
        return "\n".join(
            f"{i:03d} {self.stringify_program_line(i)}"
            for i in range(len(self.instructions))
        )

    def stringify_program_line(self, ip: int) -> str:
        inst, A, B, C = self.instructions[ip]
        # Addition:
        # addr (add register) stores into register C the result of
        # adding register A and register B.

        if inst == "addr":
            return f"register[{C}] = register[{A}] + register[{B}]"
        # addi (add immediate) stores into register C the result of
        # adding register A and value B.
        if inst == "addi":
            return f"register[{C}] = register[{A}] + {B}"

        # Multiplication:
        # mulr (multiply register) stores into register C the result
        # of multiplying register A and register B.
        if inst == "mulr":
            return f"register[{C}] = register[{A}] * register[{B}]"
        # muli (multiply immediate) stores into register C the result
        # of multiplying register A and value B.
        if inst == "muli":
            return f"register[{C}] = register[{A}] * {B}"

        # Bitwise AND:
        # banr (bitwise AND register) stores into register C the result
        # of the bitwise AND of register A and register B.
        if inst == "banr":
            return f"register[{C}] = register[{A}] & register[{B}]"
        # bani (bitwise AND immediate) stores into register C the result
        # of the bitwise AND of register A and value B.
        if inst == "bani":
            return f"register[{C}] = register[{A}] & {B}"

        # Bitwise OR:
        # borr (bitwise OR register) stores into register C the result
        # of the bitwise OR of register A and register B.
        if inst == "borr":
            return f"register[{C}] = register[{A}] | register[{B}]"
        # bori (bitwise OR immediate) stores into register C the result
        # of the bitwise OR of register A and value B.
        if inst == "bori":
            return f"register[{C}] = register[{A}] | {B}"

        # Assignment:
        # setr (set register) copies the contents of register A into register C.
        # (Input B is ignored.)
        if inst == "setr":
            return f"register[{C}] = register[{A}]"
        # seti (set immediate) stores value A into register C. (Input B is ignored.)
        if inst == "seti":
            return f"register[{C}] = {A}"

        # Greater-than testing:
        # gtir (greater-than immediate/register)
        # sets register C to 1 if value A is greater than register B.
        # Otherwise, register C is set to 0.
        if inst == "gtir":
            return f"register[{C}] = 1 if {A} > register[{B}] else 0"

        # gtri (greater-than register/immediate) sets register C to 1
        # if register A is greater than value B. Otherwise, register C is set to 0.
        if inst == "gtri":
            return f"register[{C}] = 1 if register[{A}] > {B} else 0"
        # gtrr (greater-than register/register) sets register C to 1
        # if register A is greater than register B. Otherwise, register C
        # is set to 0.
        if inst == "gtrr":
            return f"register[{C}] = 1 if register[{A}] > register[{B}] else 0"

        # Equality testing:
        # eqir (equal immediate/register) sets register C to 1
        # if value A is equal to register B. Otherwise, register C is set to 0.
        if inst == "eqir":
            return f"register[{C}] = 1 if A == register[{B}] else 0"
        # eqri (equal register/immediate) sets register C to 1
        # if register A is equal to value B. Otherwise, register C is set to 0.
        if inst == "eqri":
            return f"register[{C}] = 1 if register[{A}] == {B} else 0"
        # eqrr (equal register/register) sets register C to 1
        # if register A is equal to register B. Otherwise, register C is set to 0.
        if inst == "eqrr":
            return f"register[{C}] = 1 if register[{A}] == register[{B}] else 0"

        raise ValueError(f"Instruction line {ip}={self.instructions[ip]} unknown")

    def __repr__(self) -> str:
        return f"{self.ip=} {self.registers}"


print(Device().load(puzzle).stringify_program())
print()
print("028 register[4] = 1 if register[5] == register[0] else 0")
print(
    "The Value of rgister[5] in line 28 is the lowest non-negative integer value for register 0 that causes the program to halt after executing the fewest instructions."
)
print(
    f"Part I = {Device().load(puzzle).process_and_print_out_register_at_line(28, 5).registers[5]}"
)

000 register[5] = 123
001 register[5] = register[5] & 456
002 register[5] = 1 if register[5] == 72 else 0
003 register[2] = register[5] + register[2]
004 register[2] = 0
005 register[5] = 0
006 register[1] = register[5] | 65536
007 register[5] = 10678677
008 register[4] = register[1] & 255
009 register[5] = register[5] + register[4]
010 register[5] = register[5] & 16777215
011 register[5] = register[5] * 65899
012 register[5] = register[5] & 16777215
013 register[4] = 1 if 256 > register[1] else 0
014 register[2] = register[4] + register[2]
015 register[2] = register[2] + 1
016 register[2] = 27
017 register[4] = 0
018 register[3] = register[4] + 1
019 register[3] = register[3] * 256
020 register[3] = 1 if register[3] > register[1] else 0
021 register[2] = register[3] + register[2]
022 register[2] = register[2] + 1
023 register[2] = 25
024 register[4] = register[4] + 1
025 register[2] = 17
026 register[1] = register[4]
027 register[2] = 7
028 register[4] = 1 if register[5] == register[0

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>12935354</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>In order to determine the timing window for your underflow exploit, you also need an upper bound:</p>
<p><em>What is the lowest non-negative integer value for register <code>0</code> that causes the program to halt after executing the most instructions?</em> (The program must actually halt running forever does not count as halting.)</p>
</article>

</main>


In [48]:
# print(f"Part II = {Device(value_rigister_0=0).load(puzzle).slowest_integer()}")
# 15 minutes way too slow

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>12390302</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>


In [49]:
# from https://www.reddit.com/r/adventofcode/comments/a86jgt/2018_day_21_solutions/
r = [0] * 6
v = []
while True:
    r[3] = r[4] | 65536
    r[4] = 10678677  # user var comes from line 7 and is register[5] = 10678677
    while True:
        r[4] = r[4] + r[3] % 256
        r[4] = ((r[4] % 16777216) * 65899) % 16777216  # user var ?
        if r[3] < 256:
            break
        r[3] = r[3] // 256
    if r[4] in v:
        print(f"Part 2: {v[-1]} (after {len(v)} iterations)")
        r[0] = r[4]
    else:
        v.append(r[4])
        if len(v) == 1:
            print(f"Part 1: {v[-1]}")
    if r[4] == r[0]:
        break

Part 1: 12935354
Part 2: 12390302 (after 10916 iterations)


72